<a href="https://colab.research.google.com/github/Daniel-Benson-Poe/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/db_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd

df = pd.read_csv('https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.shape

(7043, 21)

In [3]:
# Seperate targets and features
X = df.drop(columns='Churn')
y = df['Churn']
X.shape, y.shape

((7043, 20), (7043,))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5634, 20), (5634,), (1409, 20), (1409,))

# Dataframe Exploration

In [5]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [6]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
df['Contract'].value_counts()

Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64

In [8]:
df['InternetService'].value_counts()

Fiber optic    3096
DSL            2421
No             1526
Name: InternetService, dtype: int64

In [9]:
df['MultipleLines'].value_counts()

No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

In [10]:
df['customerID'].unique().shape

(7043,)

In [11]:
df['PaymentMethod'].value_counts()

Electronic check             2365
Mailed check                 1612
Bank transfer (automatic)    1544
Credit card (automatic)      1522
Name: PaymentMethod, dtype: int64

A few things we can do to the data during cleanup:


*   Drop the customerID column - it is useless to our goal at hand
*   Convert the 'No phone service' values in the MultipleLines column to 'No'
*   Convert categorical string objects to categorical numeric





In [12]:
df.replace({"Yes": 1, "No": 0})

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,1,0,1,0,No phone service,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,Male,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,0,0,45,0,No phone service,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,1,1,24,1,1,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0
7039,2234-XADUH,Female,0,1,1,72,1,1,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0
7040,4801-JZAZL,Female,0,1,1,11,0,No phone service,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,8361-LTMKD,Male,1,1,0,4,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1


In [13]:
df['TotalCharges'].value_counts()

           11
20.2       11
19.75       9
20.05       8
19.65       8
           ..
299.7       1
1594.75     1
323.15      1
2198.3      1
73.65       1
Name: TotalCharges, Length: 6531, dtype: int64

In [14]:
df['TotalCharges'][6754]

' '

In [15]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [16]:
df['TotalCharges'][1]

'1889.5'

In [17]:
pd.to_numeric((df['TotalCharges'][0]))

29.85

In [18]:
df[df['TotalCharges'] == "0"]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [19]:
df['OnlineSecurity'].value_counts()

No                     3498
Yes                    2019
No internet service    1526
Name: OnlineSecurity, dtype: int64

In [20]:
df['OnlineBackup'].value_counts()

No                     3088
Yes                    2429
No internet service    1526
Name: OnlineBackup, dtype: int64

In [21]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

# Dataframe Wrangling

In [0]:
def wrangler(df):
  """
  Wrangles the dataframe passed into the function 
  """

  # Drop the customerID column
  df = df.drop(columns='customerID')

  # Convert the 'No phone service' values in MultipleLines to 'No'
  df['MultipleLines'] = df['MultipleLines'].replace('No phone service', 'No')

  # Convert 'No internet service' values in entire dataframe to 'No'
  df = df.replace('No internet service', 'No')

  # Convert gender from string to numeric
  df['gender'] = df['gender'].replace({'Male': 0, 'Female': 1})

  # Convert all columns containing only "Yes" or "No" values to 0s and 1s
  df = df.replace({"Yes": 1, "No": 0})

  # Convert InternetService	values from string to numeric: 'No' = 0, 'DSL' = 1, 'Fiber optic' = 2
  df['InternetService'] = df['InternetService'].replace({'No': 0, 'DSL': 1, 'Fiber optic': 2})

  # Convert Contract values from string to numeric: 'Month-to-month' = 0, 'One year' = 1, 'Two year' = 2
  df['Contract'] = df['Contract'].replace({"Month-to-month": 0, "One year": 1, "Two year": 2})

  # Convert PaymentMethod	values from string to numeric: 'Electronic check' = 0, 'Mailed check' = 1, 'Bank transfer (automatic)' = 2, 'Credit card (automatic)' = 3
  df['PaymentMethod'] = df['PaymentMethod'].replace({'Electronic check': 0, 'Mailed check': 1, 'Bank transfer (automatic)': 2, 'Credit card (automatic)': 3})

  # Convert TotalChargess " " values to "0" values
  df['TotalCharges'] = df['TotalCharges'].replace(' ', '0')

  # Convert TotalCharges values to numeric values
  df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

  # Convert TotalCharges 0 values to mean valules
  df['TotalCharges'] = df['TotalCharges'].replace(0, df['TotalCharges'].mean())

  # Convert TotalCharges values from int object to float
  df['TotalCharges'] = df['TotalCharges'].astype(float)

  # Convert all dataframe values to float
  df = df.astype(float)

  return df

In [0]:
# Function to clean target data
def target_wrangle(df):

  # Convert Yes and No values to 1 and 0 respectively
  df = df.replace({'Yes': 1, 'No': 0})
  return df

In [24]:
X_train = wrangler(X_train)
X_test = wrangler(X_test)
X_train.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2142,1.0,0.0,0.0,1.0,21.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,64.85,1336.800000
1623,1.0,0.0,0.0,0.0,54.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,1.0,2.0,97.20,5129.450000
6074,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,23.45,23.450000
1362,0.0,0.0,0.0,0.0,4.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,70.20,237.950000
6754,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,2.0,61.90,2287.087948


In [0]:
y_train = target_wrangle(y_train)
y_test = target_wrangle(y_test)

In [26]:
X_train.dtypes

gender              float64
SeniorCitizen       float64
Partner             float64
Dependents          float64
tenure              float64
PhoneService        float64
MultipleLines       float64
InternetService     float64
OnlineSecurity      float64
OnlineBackup        float64
DeviceProtection    float64
TechSupport         float64
StreamingTV         float64
StreamingMovies     float64
Contract            float64
PaperlessBilling    float64
PaymentMethod       float64
MonthlyCharges      float64
TotalCharges        float64
dtype: object

In [27]:
X_train['TotalCharges'].value_counts()

2287.087948    10
20.200000      10
19.750000       8
19.900000       6
19.550000       6
               ..
1468.900000     1
153.300000      1
8312.400000     1
4889.300000     1
74.000000       1
Name: TotalCharges, Length: 5300, dtype: int64

In [28]:
X_train['OnlineSecurity'].value_counts()

0.0    4024
1.0    1610
Name: OnlineSecurity, dtype: int64

# Preprocessing and Baseline Model

In [29]:
from sklearn.preprocessing import StandardScaler

# Instantiate Standard Scaler
scaler = StandardScaler()

# Fit_transform train data
X_train_scaled = scaler.fit_transform(X_train)

# Transform test data
X_test_scaled = scaler.transform(X_test)

X_train_scaled

array([[ 1.02516569e+00, -4.37749204e-01, -9.69578591e-01, ...,
        -2.78517888e-01, -4.73723375e-04, -4.22101828e-01],
       [ 1.02516569e+00, -4.37749204e-01, -9.69578591e-01, ...,
         5.90828251e-01,  1.07475386e+00,  1.25536334e+00],
       [-9.75452077e-01, -4.37749204e-01,  1.03137591e+00, ...,
        -1.14786403e+00, -1.37649913e+00, -1.00298825e+00],
       ...,
       [-9.75452077e-01, -4.37749204e-01,  1.03137591e+00, ...,
        -1.14786403e+00, -1.45294499e+00, -8.77996050e-01],
       [-9.75452077e-01,  2.28441306e+00, -9.69578591e-01, ...,
        -1.14786403e+00,  1.14953785e+00, -4.82541261e-01],
       [-9.75452077e-01, -4.37749204e-01, -9.69578591e-01, ...,
         1.46017439e+00, -1.49781538e+00, -8.11099129e-01]])

In [30]:
# Create our baseline model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important hyperparameters
inputs = X_train_scaled.shape[1]
epochs = 75
batch_size = 10

# Create model
model = Sequential([
    Dense(64, activation='relu', input_dim=inputs),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.fit(X_train_scaled, y_train,
          validation_data=(X_test_scaled, y_test),
          epochs=epochs,
          batch_size=batch_size
          )

Epoch 1/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4469 - accuracy: 0.7845 - val_loss: 0.4091 - val_accuracy: 0.8105
Epoch 2/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4231 - accuracy: 0.7996 - val_loss: 0.4142 - val_accuracy: 0.8048
Epoch 3/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4167 - accuracy: 0.8035 - val_loss: 0.4116 - val_accuracy: 0.7956
Epoch 4/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4117 - accuracy: 0.8074 - val_loss: 0.4057 - val_accuracy: 0.8048
Epoch 5/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4091 - accuracy: 0.8032 - val_loss: 0.4117 - val_accuracy: 0.8070
Epoch 6/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4040 - accuracy: 0.8060 - val_loss: 0.4158 - val_accuracy: 0.7984
Epoch 7/75
564/564 [==============================] - 1s 2ms/step - loss: 0.4018 - accuracy: 0.8097 - val_loss: 0.4081 - val_accuracy: 0.8091
Epoch 

# Hyperparameter Tuning

Hyperparameters to work on:

batch_size X

training epochs -- I feel like this should be the last parameter we tune. X

optimizer X

learning rate (if applicable to optimizer) X 

momentum (if applicable to optimizer) X

activation functions X

network weight initialization X

dropout regularization X

number of neurons in the hidden layer X


In [31]:
# Start by tuning the batch size

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Create model function
def create_model():

  model = Sequential([
    Dense(64, activation='relu', input_dim=inputs),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
  
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  return model

# Instantiate classifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [50]}

# Instantiate gridsearch
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Print out results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.781683874130249 using {'batch_size': 100, 'epochs': 50}
Means: 0.7520453095436096, Stdev: 0.01740295780630177 with: {'batch_size': 10, 'epochs': 50}
Means: 0.7607385993003846, Stdev: 0.007980709341759639 with: {'batch_size': 20, 'epochs': 50}
Means: 0.757368552684784, Stdev: 0.010952143771531905 with: {'batch_size': 40, 'epochs': 50}
Means: 0.7756503105163575, Stdev: 0.007614192894421179 with: {'batch_size': 60, 'epochs': 50}
Means: 0.7749376416206359, Stdev: 0.006396950461729008 with: {'batch_size': 80, 'epochs': 50}
Means: 0.781683874130249, Stdev: 0.012514893263107238 with: {'batch_size': 100, 'epochs': 50}


In [32]:
# Tune for best learning rate

# Create model function
def create_model(lr):

  optimizer = keras.optimizers.Adam(learning_rate=lr)

  model = Sequential([
    Dense(64, activation='relu', input_dim=inputs),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
  
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'lr': [0.0001, .001, .01, .1, .2, .3, .4],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.5822 - accuracy: 0.7355
Epoch 2/50
57/57 - 0s - loss: 0.5254 - accuracy: 0.7467
Epoch 3/50
57/57 - 0s - loss: 0.4967 - accuracy: 0.7600
Epoch 4/50
57/57 - 0s - loss: 0.4788 - accuracy: 0.7687
Epoch 5/50
57/57 - 0s - loss: 0.4662 - accuracy: 0.7733
Epoch 6/50
57/57 - 0s - loss: 0.4572 - accuracy: 0.7769
Epoch 7/50
57/57 - 0s - loss: 0.4505 - accuracy: 0.7827
Epoch 8/50
57/57 - 0s - loss: 0.4455 - accuracy: 0.7849
Epoch 9/50
57/57 - 0s - loss: 0.4418 - accuracy: 0.7874
Epoch 10/50
57/57 - 0s - loss: 0.4384 - accuracy: 0.7904
Epoch 11/50
57/57 - 0s - loss: 0.4359 - accuracy: 0.7906
Epoch 12/50
57/57 - 0s - loss: 0.4336 - accuracy: 0.7925
Epoch 13/50
57/57 - 0s - loss: 0.4315 - accuracy: 0.7950
Epoch 14/50
57/57 - 0s - loss: 0.4298 - accuracy: 0.7968
Epoch 15/50
57/57 - 0s - loss: 0.4282 - accuracy: 0.7985
Epoch 16/50
57/57 - 0s - loss: 0.4269 - accuracy: 0.8007
Epoch 17/50
57/57 - 0s - loss: 0.4257 - accuracy: 0.8019
Epoch 18/50
57/57 - 0s - loss: 0.4246 - 

In [35]:
# Tune for best optimizer

# Create model function
import tensorflow

def create_model(opt):

  model = Sequential([
    Dense(64, activation='relu', input_dim=inputs),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
  
  if opt == 'sgd':

    optimizer = tensorflow.keras.optimizers.SGD(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  elif opt == 'RMSprop':

    optimizer = tensorflow.keras.optimizers.RMSprop(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  elif opt == 'Adadelta':

    optimizer = tensorflow.keras.optimizers.Adadelta(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  elif opt == 'Adagrad':

    optimizer = tensorflow.keras.optimizers.Adagrad(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  elif opt == 'Nadam':

    optimizer = tensorflow.keras.optimizers.Nadam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  elif opt == 'Ftrl':

    optimizer = tensorflow.keras.optimizer.Ftrl(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'opt': ['sgd', 'RMSprop', 'Adadelta', 'Adagrad',
                      'Nadam', 'Ftrl'],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.6085 - accuracy: 0.7313
Epoch 2/50
57/57 - 0s - loss: 0.5529 - accuracy: 0.7346
Epoch 3/50
57/57 - 0s - loss: 0.5144 - accuracy: 0.7366
Epoch 4/50
57/57 - 0s - loss: 0.4865 - accuracy: 0.7551
Epoch 5/50
57/57 - 0s - loss: 0.4671 - accuracy: 0.7719
Epoch 6/50
57/57 - 0s - loss: 0.4541 - accuracy: 0.7806
Epoch 7/50
57/57 - 0s - loss: 0.4461 - accuracy: 0.7854
Epoch 8/50
57/57 - 0s - loss: 0.4411 - accuracy: 0.7911
Epoch 9/50
57/57 - 0s - loss: 0.4375 - accuracy: 0.7925
Epoch 10/50
57/57 - 0s - loss: 0.4348 - accuracy: 0.7943
Epoch 11/50
57/57 - 0s - loss: 0.4326 - accuracy: 0.7950
Epoch 12/50
57/57 - 0s - loss: 0.4309 - accuracy: 0.7953
Epoch 13/50
57/57 - 0s - loss: 0.4295 - accuracy: 0.7989
Epoch 14/50
57/57 - 0s - loss: 0.4282 - accuracy: 0.7993
Epoch 15/50
57/57 - 0s - loss: 0.4271 - accuracy: 0.7987
Epoch 16/50
57/57 - 0s - loss: 0.4260 - accuracy: 0.8014
Epoch 17/50
57/57 - 0s - loss: 0.4250 - accuracy: 0.8003
Epoch 18/50
57/57 - 0s - loss: 0.4240 - 

In [36]:
# Tune for best momentum

# Create model function
def create_model(mom):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=mom)

  model = Sequential([
    Dense(64, activation='relu', input_dim=inputs),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])
  
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'mom': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.5935 - accuracy: 0.7078
Epoch 2/50
57/57 - 0s - loss: 0.5224 - accuracy: 0.7400
Epoch 3/50
57/57 - 0s - loss: 0.4887 - accuracy: 0.7476
Epoch 4/50
57/57 - 0s - loss: 0.4689 - accuracy: 0.7630
Epoch 5/50
57/57 - 0s - loss: 0.4555 - accuracy: 0.7749
Epoch 6/50
57/57 - 0s - loss: 0.4464 - accuracy: 0.7815
Epoch 7/50
57/57 - 0s - loss: 0.4402 - accuracy: 0.7913
Epoch 8/50
57/57 - 0s - loss: 0.4360 - accuracy: 0.7945
Epoch 9/50
57/57 - 0s - loss: 0.4330 - accuracy: 0.7953
Epoch 10/50
57/57 - 0s - loss: 0.4306 - accuracy: 0.7982
Epoch 11/50
57/57 - 0s - loss: 0.4288 - accuracy: 0.7994
Epoch 12/50
57/57 - 0s - loss: 0.4272 - accuracy: 0.7998
Epoch 13/50
57/57 - 0s - loss: 0.4259 - accuracy: 0.8007
Epoch 14/50
57/57 - 0s - loss: 0.4247 - accuracy: 0.8005
Epoch 15/50
57/57 - 0s - loss: 0.4236 - accuracy: 0.8023
Epoch 16/50
57/57 - 0s - loss: 0.4226 - accuracy: 0.8010
Epoch 17/50
57/57 - 0s - loss: 0.4217 - accuracy: 0.8019
Epoch 18/50
57/57 - 0s - loss: 0.4210 - 

In [38]:
# Tune for best activation function

# Create model function
def create_model(act):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
    Dense(64, activation=act, input_dim=inputs),
    Dense(64, activation=act),
    Dense(1, activation='sigmoid')
    ])
  
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'act': ['relu', 'elu', 'exponential', 'selu', 'softmax'],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.7506 - accuracy: 0.5628
Epoch 2/50
57/57 - 0s - loss: 0.5645 - accuracy: 0.7137
Epoch 3/50
57/57 - 0s - loss: 0.5025 - accuracy: 0.7609
Epoch 4/50
57/57 - 0s - loss: 0.4715 - accuracy: 0.7797
Epoch 5/50
57/57 - 0s - loss: 0.4518 - accuracy: 0.7872
Epoch 6/50
57/57 - 0s - loss: 0.4395 - accuracy: 0.7920
Epoch 7/50
57/57 - 0s - loss: 0.4327 - accuracy: 0.7955
Epoch 8/50
57/57 - 0s - loss: 0.4290 - accuracy: 0.7939
Epoch 9/50
57/57 - 0s - loss: 0.4269 - accuracy: 0.7987
Epoch 10/50
57/57 - 0s - loss: 0.4255 - accuracy: 0.7957
Epoch 11/50
57/57 - 0s - loss: 0.4247 - accuracy: 0.7948
Epoch 12/50
57/57 - 0s - loss: 0.4240 - accuracy: 0.7953
Epoch 13/50
57/57 - 0s - loss: 0.4232 - accuracy: 0.7953
Epoch 14/50
57/57 - 0s - loss: 0.4227 - accuracy: 0.7957
Epoch 15/50
57/57 - 0s - loss: 0.4220 - accuracy: 0.7984
Epoch 16/50
57/57 - 0s - loss: 0.4218 - accuracy: 0.7987
Epoch 17/50
57/57 - 0s - loss: 0.4216 - accuracy: 0.7982
Epoch 18/50
57/57 - 0s - loss: 0.4211 - 

In [39]:
# Tune for best output activation function

# Create model function
def create_model(act):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
    Dense(64, activation='selu', input_dim=inputs),
    Dense(64, activation='selu'),
    Dense(1, activation=act)
    ])
  
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'act': ['sigmoid', 'hard_sigmoid', 'relu', 'elu', 
                      'exponential', 'selu', 'softmax'],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.6904 - accuracy: 0.5877
Epoch 2/50
57/57 - 0s - loss: 0.5677 - accuracy: 0.6986
Epoch 3/50
57/57 - 0s - loss: 0.5245 - accuracy: 0.7364
Epoch 4/50
57/57 - 0s - loss: 0.4987 - accuracy: 0.7590
Epoch 5/50
57/57 - 0s - loss: 0.4856 - accuracy: 0.7709
Epoch 6/50
57/57 - 0s - loss: 0.4844 - accuracy: 0.7772
Epoch 7/50
57/57 - 0s - loss: 0.4812 - accuracy: 0.7827
Epoch 8/50
57/57 - 0s - loss: 0.4749 - accuracy: 0.7882
Epoch 9/50
57/57 - 0s - loss: 0.4805 - accuracy: 0.7909
Epoch 10/50
57/57 - 0s - loss: 0.4830 - accuracy: 0.7941
Epoch 11/50
57/57 - 0s - loss: 0.4744 - accuracy: 0.7911
Epoch 12/50
57/57 - 0s - loss: 0.4790 - accuracy: 0.7932
Epoch 13/50
57/57 - 0s - loss: 0.4792 - accuracy: 0.7925
Epoch 14/50
57/57 - 0s - loss: 0.4822 - accuracy: 0.7964
Epoch 15/50
57/57 - 0s - loss: 0.4799 - accuracy: 0.7948
Epoch 16/50
57/57 - 0s - loss: 0.4755 - accuracy: 0.7923
Epoch 17/50
57/57 - 0s - loss: 0.4735 - accuracy: 0.7939
Epoch 18/50
57/57 - 0s - loss: 0.4767 - 

In [40]:
# Tune for best weight initializer

# Create model function
def create_model(init):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
    Dense(64, activation='selu',kernel_initializer=init ,input_dim=inputs),
    Dense(64, activation='selu', kernel_initializer=init),
    Dense(1, activation='hard_sigmoid', kernel_initializer=init)
    ])
  
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'init': ['random_normal', 'random_uniform', 'truncated_normal', 
                       'zeros', 'ones', 'glorot_normal', 'glorot_uniform'],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.8156 - accuracy: 0.5071
Epoch 2/50
57/57 - 0s - loss: 0.6182 - accuracy: 0.6642
Epoch 3/50
57/57 - 0s - loss: 0.5339 - accuracy: 0.7332
Epoch 4/50
57/57 - 0s - loss: 0.4958 - accuracy: 0.7577
Epoch 5/50
57/57 - 0s - loss: 0.4837 - accuracy: 0.7716
Epoch 6/50
57/57 - 0s - loss: 0.4737 - accuracy: 0.7788
Epoch 7/50
57/57 - 0s - loss: 0.4688 - accuracy: 0.7886
Epoch 8/50
57/57 - 0s - loss: 0.4675 - accuracy: 0.7913
Epoch 9/50
57/57 - 0s - loss: 0.4666 - accuracy: 0.7906
Epoch 10/50
57/57 - 0s - loss: 0.4750 - accuracy: 0.7916
Epoch 11/50
57/57 - 0s - loss: 0.4722 - accuracy: 0.7929
Epoch 12/50
57/57 - 0s - loss: 0.4647 - accuracy: 0.7946
Epoch 13/50
57/57 - 0s - loss: 0.4651 - accuracy: 0.7934
Epoch 14/50
57/57 - 0s - loss: 0.4647 - accuracy: 0.7945
Epoch 15/50
57/57 - 0s - loss: 0.4643 - accuracy: 0.7925
Epoch 16/50
57/57 - 0s - loss: 0.4646 - accuracy: 0.7969
Epoch 17/50
57/57 - 0s - loss: 0.4635 - accuracy: 0.7957
Epoch 18/50
57/57 - 0s - loss: 0.4633 - 

In [45]:
# Tune for best dropout layout

from tensorflow.keras.layers import Dropout

# Create model function
def create_model(input):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  if input == 1:
    model = Sequential([
    Dense(64, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
    Dropout(.2),
    Dense(64, activation='selu', kernel_initializer='glorot_uniform'),
    Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform')
    ])

  elif input == 2:
    model = Sequential([
    Dense(64, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
    Dense(64, activation='selu', kernel_initializer='glorot_uniform'),
    Dropout(.2),
    Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform')
    ])

  elif input == 3:
    model = Sequential([
    Dense(64, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
    Dropout(.2),
    Dense(64, activation='selu', kernel_initializer='glorot_uniform'),
    Dropout(.2),
    Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform'),
    ])

  elif input == 4:
    model = Sequential([
    Dense(64, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
    Dense(64, activation='selu', kernel_initializer='glorot_uniform'),
    Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform'),
    ])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'input': [1, 2, 3, 4],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 0.7099 - accuracy: 0.6031
Epoch 2/50
57/57 - 0s - loss: 0.6080 - accuracy: 0.6963
Epoch 3/50
57/57 - 0s - loss: 0.5633 - accuracy: 0.7306
Epoch 4/50
57/57 - 0s - loss: 0.5331 - accuracy: 0.7464
Epoch 5/50
57/57 - 0s - loss: 0.5009 - accuracy: 0.7607
Epoch 6/50
57/57 - 0s - loss: 0.5090 - accuracy: 0.7659
Epoch 7/50
57/57 - 0s - loss: 0.4956 - accuracy: 0.7604
Epoch 8/50
57/57 - 0s - loss: 0.5037 - accuracy: 0.7787
Epoch 9/50
57/57 - 0s - loss: 0.4996 - accuracy: 0.7767
Epoch 10/50
57/57 - 0s - loss: 0.5093 - accuracy: 0.7827
Epoch 11/50
57/57 - 0s - loss: 0.5135 - accuracy: 0.7827
Epoch 12/50
57/57 - 0s - loss: 0.4924 - accuracy: 0.7810
Epoch 13/50
57/57 - 0s - loss: 0.4955 - accuracy: 0.7817
Epoch 14/50
57/57 - 0s - loss: 0.5104 - accuracy: 0.7797
Epoch 15/50
57/57 - 0s - loss: 0.5215 - accuracy: 0.7790
Epoch 16/50
57/57 - 0s - loss: 0.5103 - accuracy: 0.7865
Epoch 17/50
57/57 - 0s - loss: 0.5096 - accuracy: 0.7820
Epoch 18/50
57/57 - 0s - loss: 0.5050 - 

In [42]:
# Tune for best dropout floating point

from tensorflow.keras.layers import Dropout

# Create model function
def create_model(drop):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
  Dense(64, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
  Dropout(drop),
  Dense(64, activation='selu', kernel_initializer='glorot_uniform'),
  Dropout(drop),
  Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform'),
  ])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'drop': [.1, .2, .3, .4, .5, .6, .7, .9, .9],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 1.2506 - accuracy: 0.5163
Epoch 2/50
57/57 - 0s - loss: 1.0740 - accuracy: 0.5410
Epoch 3/50
57/57 - 0s - loss: 0.9501 - accuracy: 0.5619
Epoch 4/50
57/57 - 0s - loss: 0.8454 - accuracy: 0.5877
Epoch 5/50
57/57 - 0s - loss: 0.7854 - accuracy: 0.6143
Epoch 6/50
57/57 - 0s - loss: 0.7095 - accuracy: 0.6338
Epoch 7/50
57/57 - 0s - loss: 0.7097 - accuracy: 0.6356
Epoch 8/50
57/57 - 0s - loss: 0.7127 - accuracy: 0.6519
Epoch 9/50
57/57 - 0s - loss: 0.6838 - accuracy: 0.6743
Epoch 10/50
57/57 - 0s - loss: 0.6632 - accuracy: 0.6908
Epoch 11/50
57/57 - 0s - loss: 0.6185 - accuracy: 0.6979
Epoch 12/50
57/57 - 0s - loss: 0.6050 - accuracy: 0.7045
Epoch 13/50
57/57 - 0s - loss: 0.6227 - accuracy: 0.7089
Epoch 14/50
57/57 - 0s - loss: 0.6376 - accuracy: 0.7188
Epoch 15/50
57/57 - 0s - loss: 0.6160 - accuracy: 0.7293
Epoch 16/50
57/57 - 0s - loss: 0.6118 - accuracy: 0.7393
Epoch 17/50
57/57 - 0s - loss: 0.6203 - accuracy: 0.7323
Epoch 18/50
57/57 - 0s - loss: 0.6074 - 

In [43]:
# Tune for best number of neruson in hidden layers

from tensorflow.keras.layers import Dropout

# Create model function
def create_model(neurons):

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
  Dense(neurons, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
  Dropout(.3),
  Dense(neurons, activation='selu', kernel_initializer='glorot_uniform'),
  Dropout(.3),
  Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform'),
  ])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'neurons': [1, 8, 16, 24, 32, 40, 48, 56, 64],
              'epochs': [50]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 1.0661 - accuracy: 0.5366
Epoch 2/50
57/57 - 0s - loss: 1.0016 - accuracy: 0.5527
Epoch 3/50
57/57 - 0s - loss: 0.9004 - accuracy: 0.5767
Epoch 4/50
57/57 - 0s - loss: 0.8433 - accuracy: 0.5932
Epoch 5/50
57/57 - 0s - loss: 0.7401 - accuracy: 0.6367
Epoch 6/50
57/57 - 0s - loss: 0.7368 - accuracy: 0.6541
Epoch 7/50
57/57 - 0s - loss: 0.7451 - accuracy: 0.6805
Epoch 8/50
57/57 - 0s - loss: 0.6511 - accuracy: 0.6951
Epoch 9/50
57/57 - 0s - loss: 0.6564 - accuracy: 0.7022
Epoch 10/50
57/57 - 0s - loss: 0.6509 - accuracy: 0.7073
Epoch 11/50
57/57 - 0s - loss: 0.6534 - accuracy: 0.7080
Epoch 12/50
57/57 - 0s - loss: 0.6488 - accuracy: 0.7153
Epoch 13/50
57/57 - 0s - loss: 0.6329 - accuracy: 0.7160
Epoch 14/50
57/57 - 0s - loss: 0.6382 - accuracy: 0.7210
Epoch 15/50
57/57 - 0s - loss: 0.6071 - accuracy: 0.7355
Epoch 16/50
57/57 - 0s - loss: 0.6116 - accuracy: 0.7297
Epoch 17/50
57/57 - 0s - loss: 0.6285 - accuracy: 0.7318
Epoch 18/50
57/57 - 0s - loss: 0.6340 - 

In [46]:
# Tune for best number of epochs

from tensorflow.keras.layers import Dropout

# Create model function
def create_model():

  optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

  model = Sequential([
  Dense(56, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
  Dropout(.3),
  Dense(56, activation='selu', kernel_initializer='glorot_uniform'),
  Dropout(.3),
  Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform'),
  ])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

model = KerasClassifier(build_fn=create_model, verbose=2)

# Define the grid search parameters
param_grid = {'batch_size': [100],
              'epochs': [10, 20, 30, 40, 50, 100, 200, 300]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

# Report results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Epoch 1/50
57/57 - 0s - loss: 1.4307 - accuracy: 0.4512
Epoch 2/50
57/57 - 0s - loss: 1.1654 - accuracy: 0.4870
Epoch 3/50
57/57 - 0s - loss: 0.9403 - accuracy: 0.5501
Epoch 4/50
57/57 - 0s - loss: 0.8241 - accuracy: 0.5863
Epoch 5/50
57/57 - 0s - loss: 0.7936 - accuracy: 0.6354
Epoch 6/50
57/57 - 0s - loss: 0.7510 - accuracy: 0.6409
Epoch 7/50
57/57 - 0s - loss: 0.7209 - accuracy: 0.6608
Epoch 8/50
57/57 - 0s - loss: 0.6712 - accuracy: 0.6773
Epoch 9/50
57/57 - 0s - loss: 0.6522 - accuracy: 0.6876
Epoch 10/50
57/57 - 0s - loss: 0.6848 - accuracy: 0.7133
Epoch 11/50
57/57 - 0s - loss: 0.6313 - accuracy: 0.7123
Epoch 12/50
57/57 - 0s - loss: 0.6535 - accuracy: 0.7190
Epoch 13/50
57/57 - 0s - loss: 0.6428 - accuracy: 0.7270
Epoch 14/50
57/57 - 0s - loss: 0.6120 - accuracy: 0.7412
Epoch 15/50
57/57 - 0s - loss: 0.6264 - accuracy: 0.7364
Epoch 16/50
57/57 - 0s - loss: 0.6242 - accuracy: 0.7386
Epoch 17/50
57/57 - 0s - loss: 0.6089 - accuracy: 0.7437
Epoch 18/50
57/57 - 0s - loss: 0.5928 - 

# Final Product

In [47]:
# Final Product

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Important hyperparameters
inputs = X_train_scaled.shape[1]
epochs = 50
batch_size = 100
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001, momentum=0.1)

# Create model
model = Sequential([
    Dense(56, activation='selu',kernel_initializer='glorot_uniform' ,input_dim=inputs),
    Dropout(.3),
    Dense(56, activation='selu',kernel_initializer='glorot_uniform'),
    Dropout(.3),
    Dense(1, activation='hard_sigmoid', kernel_initializer='glorot_uniform')
    ])

# Compile model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Fit model
model.fit(X_train_scaled, y_train,
          validation_data=(X_test_scaled, y_test),
          epochs=epochs,
          batch_size=batch_size
          )

Epoch 1/50
57/57 [==============================] - 0s 4ms/step - loss: 1.5528 - accuracy: 0.5584 - val_loss: 0.6855 - val_accuracy: 0.6139
Epoch 2/50
57/57 [==============================] - 0s 2ms/step - loss: 1.1799 - accuracy: 0.5829 - val_loss: 0.6156 - val_accuracy: 0.6891
Epoch 3/50
57/57 [==============================] - 0s 2ms/step - loss: 1.0550 - accuracy: 0.6203 - val_loss: 0.5778 - val_accuracy: 0.7175
Epoch 4/50
57/57 [==============================] - 0s 3ms/step - loss: 0.9606 - accuracy: 0.6349 - val_loss: 0.5561 - val_accuracy: 0.7317
Epoch 5/50
57/57 [==============================] - 0s 2ms/step - loss: 0.9113 - accuracy: 0.6443 - val_loss: 0.5369 - val_accuracy: 0.7445
Epoch 6/50
57/57 [==============================] - 0s 2ms/step - loss: 0.8483 - accuracy: 0.6580 - val_loss: 0.5122 - val_accuracy: 0.7551
Epoch 7/50
57/57 [==============================] - 0s 2ms/step - loss: 0.8051 - accuracy: 0.6786 - val_loss: 0.4978 - val_accuracy: 0.7672
Epoch 8/50
57/57 [==

Perfecct! We reached a val_accuracy above 81% in 50 epochs using the tuned parameters above.

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?